### CS4423 - Networks
Angela Carnevale <br />
School of Mathematical and Statistical Sciences<br />
NUI Galway

#### 6. Directed Networks

# Week 11, lecture 2: The Structure of the World Wide Web

We have seen that in some networks, especially information networks, edges carry an additional piece of information: they tell us which node is the **source** and which is the **target**. That is, they are **directed edges** (we can think of them as arrows). 

As with undirected graphs, an interesting question in
directed graphs is: which nodes can be reached from
a given node?

### Reachability in Directed Graphs

Recall that a **directed graph** is a pair $G = (X, E)$
with **vertex set** $X$  and **edge set** $E \subseteq X^2 = X \times X$.
For an edge $(x, y) \in E$ we sometimes write $x \to y$.

A **path** in a directed graph  $G = (X, E)$
is a sequence of nodes $(x_0, x_1, \dots, x_l)$
with $x_{i-1} \to x_i$ for $i = 1,\dots, l$.
The number $l$ is called the **length** of the path.
We write $x \leadsto y$
if there exists a path (possibly of length $0$)
from $x$ to $y$ in $G$.

**Definition.** A directed graph $G$ is **weakly connected** if, when
considerd as undirected graph, it is connected.
The **weakly connnected components** (WCCs) of $G$ are its connected components,
when considered as undirected graph.

**Definition.** A directed graph $G$ is **strongly connected** if, for
each pair of vertices $x, y \in X$, there is a path from
$x$ to $y$ in $G$, i.e., if $x \leadsto y$.

A **strongly connected component (SCC)** of a directed graph $G$
is a subset $C$ of $X$ which is 

(i) strongly connected, and

(ii) not part of a larger strongly connected subset of $X$.

In general, a directed graph is a collection of WCCs.
Each WCC in turn is a collection of SCCs.

When a directed graph $G$ is regarded as a **relation**
on the set $X$, strongly connected components can be described as
the **equivalence classes** of an equivalence relation that is obtained
as follows.

First note that the relation ${x \leadsto y}$
is the reflexive and transitive closure of the
edge relation $x \to y$.  

Thus, by construction it is reflexive and
transitive.  

It needn't be symmetric, and it might not be anti-symmetric, though. That is, there might be vertices $x$ and $y$
with $x\leadsto y$ and $y 
\leadsto x$ (and it won't be all of them).

However, we can define a new relation as follows.

**Definition.** We set $x \equiv y$ if $x \leadsto y$ _and_ $y \leadsto x$.

This **is** an equivalence relation we get equivalence classes that partition our graph. These equivalence classes are the strongly connected
components of $G$.  We denote the class of $x \in X$ by $[x]$.

Moreover, there is a partial order relation
$\leq$ (a relation which is reflexive, transitive and anti-symmetric)
on the set of equivalence classes:

$[x] \leq [y]$ if $x \leadsto y$.

We can say something about the structure of the WWW in terms of these equivalence classes and of the partial order on them.

### The Bow-Tie Structure of the WWW

Like the giant component in a simple graph, it turns out that
a directed graph with sufficiently many edges
has  a **giant SCC**.

The remainder of the graph consists of four more sets of components of nodes, as follows:

1. IN: upstream components, the set of all components
$C$ with $C <$ SCC.

2. OUT: downstream components,
the set of all components $C$ with $C >$ SCC.

3. tendrils: the set of all components $C$ with either $C >$ IN and $C \not<$ OUT
or $C <$ OUT and $C \not>$ IN; <BR />
and tubes: components $C$ with $C >$ IN, $C <$ OUT but $C \not <$ SCC.

4. disconnected components.

Thus, in any directed graph with a distinguished SCC,
the WCC in which it is contained
necessarily has the following global bow-tie structure:

![bow tie](images/bowtie.png)

Research on available data on the Web in 1999 has confirmed this
bow tie structure for the World Wide Web, with a **large Giant SCC**
covering less than $\frac13$ of the vertex set,
and the
three parts IN, OUT and the tendrils and tubes roughly of the same
size.  One can assume that this proportion has not changed much over
time, although the advent of social media
has brought many new types of links and
documents to the Web.


## Computing Bow-Tie Components

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
opts = { "with_labels" : True, "node_color" : 'y' }

**Example.**  Let's start with a reasonably large random **directed graph**,
using the Erdős-R&eacute;nyi $G(n, m)$ model:

In [ ]:
n, m = 100, 120
G = nx.gnm_random_graph(n, m, directed=True)

### Weakly Connected Components

The weakly connected components of a directed graph $G$ can be determined by BFS, as before,
counting as "neighbors" of a node $x$ **both** its _successors_ and it _predecessors_ in the graph.

A single component, the weakly connected component of node $x$, is found as follows.

In [ ]:
def weak_component(G, x):
    nodes = {x}
    queue = [x]
    for y in queue:
        G.nodes[y]["seen"] = True
        for z in set(G.successors(y)) | set(G.predecessors(y)): ## preds+succs are the neighbours of a node
            if z not in nodes:
                nodes.add(z)
                queue.append(z)
    return nodes

The list of all weakly connected components is computed by looping over all the  nodes of `G`,
computing the components of "unseen" nodes and collecting them in a list.
The final result is sorted by decreasing length before it is returned.

In [ ]:
def weak_components(G):
    
    # initialize
    wccs = []
    
    # find each node's wcc
    for x in G:
        if not G.nodes[x].get("seen"):
            wccs.append(weak_component(G, x))
            
    # clean up after yourself
    for x in G:
        del G.nodes[x]["seen"]
        
    # return sorted list of wccs
    return sorted(wccs, key=len, reverse=True)

In [ ]:
wccs = weak_components(G)
len(wccs)

In [ ]:
[len(c) for c in wccs]

### Strongly Connected Components

**Strongly** connected components are efficiently found by DFS.
[Tarjan's Algorithm](https://en.wikipedia.org/wiki/Tarjan%27s_strongly_connected_components_algorithm) cleverly
uses recursion and an additional stack for this.

The following function finds strongly connected components in a recursive fashion. 

In [ ]:
def connect(G, stack, sccs, idx, x):
    G.nodes[x]["low"] = G.nodes[x]["idx"] = idx
    idx += 1
    stack.append(x)
    G.nodes[x]["stacked"] = True
    for y in G[x]:
        if "idx" not in  G.nodes[y]:
            idx = connect(G, stack, sccs, idx, y)
            G.nodes[x]["low"] = min(G.nodes[x]["low"], G.nodes[y]["low"])
        elif G.nodes[y]["stacked"]:
            G.nodes[x]["low"] = min(G.nodes[x]["low"], G.nodes[y]["idx"])
                
    if G.nodes[x]["low"] == G.nodes[x]["idx"]:
        scc = []
        while True:
            y = stack.pop()
            G.nodes[y]["stacked"] = False
            scc.append(y)
            if y == x:
                break
        sccs.append(scc)
            
    return idx


Similar as in the case of the weakly connected components, the overall algorithm
uses a loop over all the nodes of `G` to find all strongly connected components.

In [ ]:
def strong_components(G):
    
    # initialize
    idx = 0
    stack = []
    sccs = []
    
    # find each node's scc
    for x in G:
        if "idx" not in G.nodes[x]:
            idx = connect(G, stack, sccs, idx, x)

    # clean up after yourself
    for x in G:
        del G.nodes[x]["idx"]
        del G.nodes[x]["low"]
        del G.nodes[x]["stacked"]
    
    # return sorted list of sccs
    return sorted(sccs, key = len, reverse=True)

In [ ]:
sccs = strong_components(G)
[len(c) for c in sccs[:10]] ## this list is eventually 1 so looking at a few entries suffices

As the resulting list of components is sorted by length, in descending order,
`sccs[0]` is the **Giant SCC**.

### The OUT Components

The components reachable from the Giant can be found by BFS applied to any
vertex in the Giant. So here is a node representating the Giant:

In [ ]:
rep = min(sccs[0])
rep

This variant of BFS considers the `successors` of a node $x$ as its neighbors.

In [ ]:
def outreach(G, x):
    """find all nodes that can be reached 
    from node x in the directed graph G"""
    nodes = {x}
    queue = [x]
    for y in queue:
        for z in G.successors(y):
            if z not in nodes:
                nodes.add(z)
                queue.append(z)
    return nodes

The resulting list of nodes consists of the Giant and the OUT components.

In [ ]:
out = outreach(G, rep)
len(out)

The components reachable from the Giant can be found by BFS applied to any
vertex in the Giant, following the arrows in reverse. 

This variant of BFS considers the `predecessors` of a node $x$ as its neighbors.

In [ ]:
def innreach(G, x):
    """find all nodes that can reach 
    node x in the directed graph G"""
    nodes = {x}
    queue = [x]
    for y in queue:
        for z in G.predecessors(y):
            if z not in nodes:
                nodes.add(z)
                queue.append(z)
    return nodes

The resulting list of nodes consists of the Giant and the IN components.

In [ ]:
inn = innreach(G, rep)
len(inn)

The Giant is the intersection of `inn` and `out`. 

In [ ]:
giant = inn & out

Let's call the union of `inn` and `out` the **core** of the graph `G`.

In [ ]:
core = inn | out
len(core)

And let's remove the Giant from `inn` and `out`:

In [ ]:
inn1 = inn - out
out1 = out - inn
len(giant), len(inn1), len(out1)

## Code Corner

### `python`

* `setX & setY`, `setX | setY`, `setX - setY`: [[doc]](https://docs.python.org/2/library/stdtypes.html#set) set operations, intersection, union, difference


* `x in setY`, `x not in setY`: set membership


* `setX.issubset(setY)`, `setX <= setY`, `setX < setY`: subset relationship